# Init

In [1]:
%load_ext autoreload

In [2]:
from app import app
context = app.app_context()
context.push()

ModuleNotFoundError: No module named 'app'

## Imports

In [ ]:
from develop.utils import formatters

## Inputs

In [ ]:
query="What is the relationship between Zn2+ and glycolate?"

## Initial check

Current response:

In [ ]:
import openai
response = openai.ChatCompletion.create(
    messages=[
        { "role": "user", "content": query }
    ],
    model="gpt-3.5-turbo",
    temperature=0
)
response

# Compose chain

## Init llm model

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, verbose=True)
llm

## Pepare prompts for core term extraction

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
QUERY_KEY = 'query'

core_terms_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            template="Always response with comma separated list of chemical or biological terms identified in prompt.",
            input_variables=[]
        )
    ),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=f"{{{QUERY_KEY}}}",
            input_variables=[QUERY_KEY],
        )
    )
])
core_terms_prompt_template

## Define embedding model

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
embeddings

## Define vectorstore

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
# Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db_oai")
vectorstore

## Get graph reference

### Define neo4j connector

In [ ]:
from llmlib.database import Neo4j

In [ ]:
graph = Neo4j().graph()
graph.schema = ''  # Overwrite schema to avoid it to be loaded from db (lengthy process not ussed in here)
graph

#### Init graph search API wrapper

In [ ]:
from llmlib.utils.search.cypher_search_api_wrapper import CypherSearchAPIWrapper

In [ ]:
search = CypherSearchAPIWrapper(
    graph=graph,
    verbose=True,
    return_intermediate_steps=True
)
search

## Compose QA chain with graph retriever

### Init graph retriever

In [ ]:
from llmlib.utils.retrievers.graph_search_retriever import GraphSearchRetriever

In [ ]:
retriever = GraphSearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    graph_search=search,
    prompt=core_terms_prompt_template,
    verbose=True,
    return_intermediate_steps=True
)
retriever

### Define callback handler

In [ ]:
from langchain.callbacks.stdout import StdOutCallbackHandler
handler = StdOutCallbackHandler()
handler

### Init QA chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
    callbacks=[handler]
)
qa

# Test

## Initial query

In [ ]:
qa.run(query, callbacks=[handler])

## Check background noise

In [ ]:
qa.run("What is the relationship between INHBA and MTMR4?", callbacks=[handler])

## Check different qa stuffing techniques

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    verbose=True,
    callbacks=[handler]
)
qa

In [ ]:
qa.run(query, callbacks=[handler])

In [ ]:
qa.run("What is the relationship between INHBA and MTMR4?", callbacks=[handler])